In [11]:
import requests
from bs4 import BeautifulSoup
import re
import json
from config.config import BASE_URL, OUTPUT_FILE
from logger.logger import logger

headers = {'User-Agent': 'Mozilla/5.0'}

def get_unique_item_ids(max_pages=1):
    """Fetch unique item IDs from paginated pages with proper logging."""
    item_ids = set()
    page = 1
    has_more_pages = True
    
    logger.info(f"Starting to scrape item IDs (max pages: {max_pages})")
    
    while has_more_pages:
        if max_pages and page > max_pages:
            logger.info(f"Reached max pages limit ({max_pages})")
            break
            
        url = f"{BASE_URL}?page={page}"
        logger.debug(f"Fetching page {page}: {url}")
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                logger.error(f"Failed to fetch page {page}. Status code: {response.status_code}")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            
            # Find all item links and extract IDs from hrefs
            current_page_ids = set()
            for a in soup.select("a.products-i__link"):
                href = a.get("href")
                if href:
                    match = re.search(r'/autos/(\d+)', href)
                    if match:
                        item_id = match.group(1)
                        current_page_ids.add(item_id)
            
            # Check if we found any new items on this page
            if not current_page_ids:
                logger.info(f"No items found on page {page}, stopping pagination")
                has_more_pages = False
            else:
                new_items = current_page_ids - item_ids
                if not new_items:
                    logger.info(f"No new items found on page {page}, stopping pagination")
                    has_more_pages = False
                else:
                    logger.info(f"Found {len(new_items)} new items on page {page}")
                    item_ids.update(current_page_ids)
                    page += 1
                    
        except requests.RequestException as e:
            logger.error(f"Network error fetching page {page}: {str(e)}", exc_info=True)
            break
        except Exception as e:
            logger.error(f"Unexpected error processing page {page}: {str(e)}", exc_info=True)
            break
    logger.info(f"Finished scraping. Found {len(item_ids)} unique items")
    return item_ids    

# Run the function and print results
item_ids = get_unique_item_ids()
links = list(item_ids)
print(links)
print(len(links))

2025-06-05 14:16:13 - scraper - INFO - Starting to scrape item IDs (max pages: 1)
2025-06-05 14:16:15 - scraper - INFO - Found 36 new items on page 1
2025-06-05 14:16:15 - scraper - INFO - Reached max pages limit (1)
2025-06-05 14:16:15 - scraper - INFO - Finished scraping. Found 36 unique items


['9475914', '9484809', '8962955', '9479629', '9477675', '9459989', '9483665', '9410138', '9437839', '9514140', '9514137', '9513319', '9055483', '8977140', '9403912', '9316501', '9480119', '9442779', '9493572', '9495617', '9315607', '9478943', '9467401', '9514138', '9474846', '9350637', '9497122', '9315428', '9165427', '9304577', '9514142', '9372756', '9427264', '9512985', '9204187', '8948342']
36


In [12]:
def extract_item_data(link):
    """Extract detailed item data from individual product page with comprehensive logging"""
    url = "https://turbo.az/autos/" + link
    logger.debug(f"Starting to extract data from: {url}")
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            logger.error(f"Failed to fetch {url} - Status code: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract item ID
        item_id = link.split("/")[-1]
        logger.debug(f"Processing item ID: {item_id}")

        # Extract amount and currency using meta tags
        meta_tag = soup.find('meta', {'property': 'og:title'})

        # Extract content attribute
        content = meta_tag.get('content')

        # Use regex to find price followed by either USD or AZN (case-insensitive)
        match = re.search(r'(\d[\d\s]*\d)\s+(USD|AZN)', content, re.IGNORECASE)

        if match:
            amount = match.group(1).replace(' ', '')  # Remove space
            currency = match.group(2).upper()  # Get the currency type

        logger.debug(f"Extracted price: {amount} {currency}")

        # Extract property fields dynamically
        properties = {}
        prop_div = soup.find("div", class_="product-properties__column")
        if prop_div:
            for prop in prop_div.find_all("div", class_="product-properties__i"):
                try:
                    name = prop.find("label", class_="product-properties__i-name").get_text(strip=True)
                    value = prop.find("span", class_="product-properties__i-value").get_text(strip=True)
                    properties[name] = value
                    logger.debug(f"Found property: {name} = {value}")
                except Exception as e:
                    logger.warning(f"Failed to extract property: {str(e)}", exc_info=True)

        # Construct result dictionary
        item_data = {
            'item_id': item_id,
            'amount': amount,
            'currency': currency,
        }

        # Add all found properties dynamically
        item_data.update(properties)
        
        logger.info(f"Successfully processed item: {item_id}")
        return item_data

    except requests.RequestException as e:
        logger.error(f"Network error while processing {url}: {str(e)}", exc_info=True)
        return None
    except Exception as e:
        logger.error(f"Unexpected error processing {url}: {str(e)}", exc_info=True)
        return None


def scrape_all_items():
    """Main scraping function that processes all items with progress tracking"""
    if not links:
        logger.warning("No links to process - empty list provided")
        return

    results = []
    logger.info(f"Starting to process {len(links)} items")

    for idx, link in enumerate(links, 1):
        logger.info(f"[{idx}/{len(links)}] Processing: {link}")
        data = extract_item_data(link)
        if data:
            results.append(data)
            logger.debug(f"Added item {data.get('item_id')} to results")
        else:
            logger.warning(f"Failed to process item: {link}")

    # Save to JSON
    try:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        logger.info(f"Successfully saved {len(results)} items to {OUTPUT_FILE}")
        if len(results) != len(links):
            logger.warning(f"Processed {len(results)} out of {len(links)} items ({(len(results)/len(links))*100:.1f}% success rate)")
    except Exception as e:
        logger.error(f"Failed to save data to {OUTPUT_FILE}: {str(e)}", exc_info=True)


# Run the scraper
scrape_all_items()

2025-06-05 14:16:17 - scraper - INFO - Starting to process 36 items
2025-06-05 14:16:17 - scraper - INFO - [1/36] Processing: 9475914


2025-06-05 14:16:17 - scraper - INFO - Successfully processed item: 9475914
2025-06-05 14:16:17 - scraper - INFO - [2/36] Processing: 9484809
2025-06-05 14:16:18 - scraper - INFO - Successfully processed item: 9484809
2025-06-05 14:16:18 - scraper - INFO - [3/36] Processing: 8962955
2025-06-05 14:16:19 - scraper - INFO - Successfully processed item: 8962955
2025-06-05 14:16:19 - scraper - INFO - [4/36] Processing: 9479629
2025-06-05 14:16:19 - scraper - INFO - Successfully processed item: 9479629
2025-06-05 14:16:19 - scraper - INFO - [5/36] Processing: 9477675
2025-06-05 14:16:20 - scraper - INFO - Successfully processed item: 9477675
2025-06-05 14:16:20 - scraper - INFO - [6/36] Processing: 9459989
2025-06-05 14:16:20 - scraper - INFO - Successfully processed item: 9459989
2025-06-05 14:16:20 - scraper - INFO - [7/36] Processing: 9483665
2025-06-05 14:16:21 - scraper - INFO - Successfully processed item: 9483665
2025-06-05 14:16:21 - scraper - INFO - [8/36] Processing: 9410138
2025-0